In [2]:
import pandas as pd
import numpy as np

In [ ]:
# Read in the data from a JSON file
df = pd.read_json('indeed-data-jobs-FINAL.json')
df.sample(5)

In [ ]:
# these were probably removed so I'll remove them from the data
df[df.job_description == '']

In [ ]:
# REMOVED ROWS ABOVE
df = df[df.job_description != ''].reset_index(drop=True)
df

In [ ]:
# rows, columns
df.shape

In [ ]:
# data summary
df.info()

# Creating Labels
### I'll need to create a script that uses the job title to create the label
I already see that data entry isn't what I'm looking for so I'll need to iterate over all entries to purge jobs like this that don't fall into 1 of my 4 categories

In [ ]:
df.job_title

In [ ]:
pd.DataFrame(df.loc[0]).T

In [ ]:
### Regex

In [ ]:
df

## [FAIL] Finding each Label by Indexing
### The data is sort of in order by the keyword I used while scraping

- 1st: 'data scientist'
- 2nd: 'data analyst'
- 3rd: 'data engineer'
- 4th: 'machine learning engineer'

*I scraped about the same amount of jobs for each of the 4 labels. So I'm going to index the dataframe for the range within each quartile because I know that the job titles will change around that point so I'll just differentiate the labels by peeking into the DataFrame from the reference points.*

In [ ]:
print(f'1st quarter: {898*.25} \n 2nd quarter: {898*.5} \n 3rd quarter: {898*.75}')

In [ ]:
# the last data scientist entry was at 223
len(df[:224])

In [ ]:
# the last data analyst entry was at 447
len(df[224:447])

In [ ]:
# the last data engineer entry was at 676
len(df[447:674])

In [ ]:
# the rest of the jobs are machine learning engineers
len(df[674:])

In [ ]:
ds = ['DS' for n in range(0,224)]
da = ['DA' for n in range(0,223)]
de = ['DE' for n in range(0,227)]
mle = ['MLE' for n in range(0,224)]

labels = ds + da + de + mle
len(labels)

In [ ]:
# Add labels column from the labels list
df['label'] = labels

In [ ]:
#peek at sample to see if code is effective
df[['job_title', 'label']].sample(5)

In [ ]:
import re
# checking to be sure any stray jobs that aren't a target be removed
df['valid'] = df.job_title.str.lower().str.extract('(.*?data.*?scientist.*?|.*?data.*?analyst.*?|data.*?engineer.*?|.*?machine.*?learning.*?engineer.*?)')

In [ ]:
# 58 nulls
null_rows = df[df.valid.isnull()]
null_rows

### Clean up the rest of the Labels (58)

The data scientist entries dont seem to be true data scientist but data entry or research scientist so I'll drop those (4)


In [ ]:
# delete
ds_drop = null_rows[0:4]

In [ ]:
ds_drop

Remove the engineer positions that don't correspond to the work of a data engineer (2)

In [ ]:
# delete
de_drop = null_rows[null_rows.label == "DE"]

In [ ]:
de_drop

Going to remove titles that don't have 'analyst included, like data specialist which is essentially a different position (regex for analyst)

In [ ]:
# Keep
mask = []

for row in null_rows.job_title:
    if re.search(r'analyst', row, flags=re.IGNORECASE) != None:
        mask.append(True)
    else:
        mask.append(False)

In [ ]:
da_keep = null_rows[mask]

As with the data analyst, most position are in fact mle's but I'll use regex to keep 'learning engineer' and 'machine learning' to filter out the rest.

In [ ]:
# keep 
mask = []

for row in null_rows.job_title:
    if re.search(r'machine learning|learning engineer', row, flags=re.IGNORECASE) != None:
        mask.append(True)
    else:
        mask.append(False)

In [ ]:
mle_keep = null_rows[mask]

### Delete rows

In [ ]:
rows_to_keep = null_rows.drop(ds_drop.index).drop(de_drop.index)
print(len(rows_to_keep))

In [ ]:
# drop original 58 that needed to be validated
df = df.drop(null_rows.index)

In [ ]:
# merge the rows that we decided to keep and drop the valid column
df = pd.concat([df, rows_to_keep]).drop(columns='valid')

In [ ]:
df.info()

In [ ]:
df.sample(10)

### Looks better!

In [4]:
from prepare import prep_create_labels
prep_df = prep_create_labels(pd.read_json('indeed-data-jobs-FINAL.json'))

In [ ]:
(prep_df == df).sum()

In [8]:
prep_df.sample(5)

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description,label
461,Data Engineer,"USG, Inc","San Antonio, TX",1,$50 - $55 an hour,14 days ago,2021-03-05,Looking for Consultants who can work on our W2...,DE
556,Sr Data Engineer,eProSoft America Inc,"Las Vegas, NV 89119",1,Up to $65 an hour,23 days ago,2021-03-05,Looking for a very strong Data Engineer for a ...,DE
621,Data & Reporting Engineer,Transcendent Solutions LLC,"Oldsmar, FL 34677",0,,7 days ago,2021-03-05,Description: The Mission of the Role Transcen...,DE
548,Data Engineer I or II - Can Be Remote Based On...,Associated Bank,"Milwaukee, WI",1,,14 days ago,2021-03-05,Associated Bank is an equal opportunity employ...,DE
849,Machine Learning Application Engineer in Healt...,Oak Ridge National Laboratory,"Oak Ridge, TN 37831",0,,7 days ago,2021-03-05,Requisition Id 5187 Overview and Purpose: We a...,MLE
